## ЛР 2. Извлечение именованных сущностей

In [145]:
import wikipedia
import re
from yargy import Parser, rule, and_, or_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, dictionary, gte, lte, eq, type as yatype
from yargy.pipelines import pipeline, morph_pipeline
from IPython.display import display

wikipedia.set_lang("ru")


### Загрузим статью из Википедии

In [146]:
TITLE = 'Франция'

wiki = wikipedia.page(TITLE)
text = wiki.content
text = re.sub(r'==.*?==+', '', text)
len(text.split())

12731

### Сохраним статью в файл

In [147]:
FILENAME = 'france.txt'

f = open(FILENAME, "w")
f.write(text)
f.close()

### Прочитаем файл

In [148]:
f = open(FILENAME, "r")
text = f.read()
sents = text.splitlines()
len(sents)

605

In [149]:
sents[:3]

['Фра́нция (фр. France, [fʁɑ̃s]  слушать), официальное название — Францу́зская Респу́блика (фр. République française, [ʁe.py.blik fʁɑ̃.sɛz]  слушать) — трансконтинентальное государство, включающее основную территорию в Западной Европе и ряд заморских регионов и территорий. Столица — Париж. Девиз Республики — «Свобода, равенство, братство», её принцип — правление народа, народом и для народа.',
 'Название страны происходит от этнонима древнегерманского племени франков. Большинство населения Франции имеет смешанное галло-романское происхождение и говорит на языке романской группы.',
 'Население — 62 814 233 человека в метрополии и 67 848 156 человек — с учётом заморских владений (оценка на 1 июля 2020), в том числе около 90 % — граждане Франции. Состав населения Франции по вероисповеданию по состоянию на 2019 год: католики — 41 %, нерелигиозные — 40 %, мусульмане — 5 %, другие религии — 5 %, лютеране — 2 %, православные — 2 %, другие христиане — 2 %, буддисты — 1 % , иудеи — 1 %, нет дан

In [150]:
# Применение парсера
def parse(rules):
    parser = Parser(rules)
    for sent in sents:
        for match in parser.findall(sent):
            display(match.fact)

## Определение именованных сущностей

### Персоны с должностями

In [151]:
Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)


POSITION = morph_pipeline([
    'президент',
    'король'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parse(PERSON)

Person(
    position='президент',
    name=Name(
        first='жак',
        last='ширак'
    )
)

### Полные даты

In [152]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'май',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}


MONTH_NAME = dictionary(MONTHS)
DAY = and_(
    gte(1),
    lte(31)
)
YEAR = and_(
    gte(400),
    lte(2100)
)
DATE = rule(
    DAY.interpretation(
        Date.day
    ),
    MONTH_NAME.interpretation(
        Date.month
    ),
    YEAR.interpretation(
        Date.year
    ).optional()
).interpretation(
    Date
)

parse(DATE)

Date(
    year='2020',
    month='июля',
    day='1'
)

Date(
    year='1970',
    month='ноября',
    day='9'
)

Date(
    year='1995',
    month='мая',
    day='17'
)

Date(
    year='2007',
    month='мая',
    day='16'
)

Date(
    year='2007',
    month='мая',
    day='16'
)

...

Date(
    year='2021',
    month='апреля',
    day='14'
)

Date(
    year=None,
    month='июня',
    day='21'
)

Date(
    year='2021',
    month='апреля',
    day='14'
)

Date(
    year='2021',
    month='апреля',
    day='14'
)

Date(
    year='2021',
    month='апреля',
    day='14'
)

### Названия в кавычках

In [153]:
Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)

TITLE = rule(
    '«',
    not_(eq('»')).repeatable(),
    '»'
)
ITEM = rule(
    TITLE.interpretation(
        Item.titles
    ),
    eq(',').optional()
).repeatable().interpretation(
    Item
)

parse(ITEM)

Item(
    titles=['«Свобода, равенство, братство»']
)

Item(
    titles=['«Франция»']
)

Item(
    titles=['«Франкия»']
)

Item(
    titles=['«страна франков»']
)

Item(
    titles=['«франки»']
)

...

Item(
    titles=['«Les Echos»']
)

Item(
    titles=['«20 minutes»']
)

Item(
    titles=['«Direct matin»']
)

Item(
    titles=['«Metro»']
)

Item(
    titles=['«Ouest-France»']
)

### Описание страны через прилагательные

In [154]:
Country = fact(
    'Country',
    ['adj', 'subj']
)

COUNTRY = rule(
    rule(
        gram('ADJF').repeatable()
    ).interpretation(Country.adj),
    rule(
        dictionary({'страна', 'Франция', 'государство'})
    ).interpretation(Country.subj)
).interpretation(Country)

parse(COUNTRY)

Country(
    adj='трансконтинентальное',
    subj='государство'
)

Country(
    adj='Франкское',
    subj='государство'
)

Country(
    adj='Франкского',
    subj='государства'
)

Country(
    adj='современной',
    subj='Франции'
)

Country(
    adj='Франкское',
    subj='государство'
)

...

Country(
    adj='других',
    subj='стран'
)

Country(
    adj='Южной',
    subj='Франции'
)

Country(
    adj='средневековой',
    subj='Франции'
)

Country(
    adj='сборная',
    subj='Франции'
)

Country(
    adj='Сборная',
    subj='Франции'
)

### Количественные показатели (в процентах)

In [155]:
Percent = fact(
    'Percent',
    ['count', 'subj']
)

PERCENT = rule(
    rule(
        yatype('INT'),
        '%'
    ).interpretation(Percent.count),
    gram('NOUN').interpretation(Percent.subj)
).interpretation(Percent)

parse(PERCENT)

Percent(
    count='30 %',
    subj='населения'
)

Percent(
    count='10 %',
    subj='населения'
)

Percent(
    count='2 %',
    subj='населения'
)

Percent(
    count='2 %',
    subj='территории'
)

Percent(
    count='27 %',
    subj='территории'
)

...

Percent(
    count='82 %',
    subj='территории'
)

Percent(
    count='30 %',
    subj='территории'
)

Percent(
    count='14 %',
    subj='стоимости'
)

Percent(
    count='64 %',
    subj='расходов'
)

Percent(
    count='40 %',
    subj='времени'
)

### Количественные показатели с учетом грамматики (в процентах)

In [156]:
Percent = fact(
    'Percent',
    ['count', 'subj']
)

PERCENT = rule(
    or_(
        gram('NOUN'),
        gram('ADJF')
    ).interpretation(Percent.subj),
    '—',
    rule(
        yatype('INT'),
        '%'
    ).interpretation(Percent.count),
    gram('NOUN').optional().interpretation(Percent.subj)
).interpretation(Percent)

parse(PERCENT)

Percent(
    count='41 %',
    subj='католики'
)

Percent(
    count='40 %',
    subj='нерелигиозные'
)

Percent(
    count='5 %',
    subj='мусульмане'
)

Percent(
    count='5 %',
    subj='религии'
)

Percent(
    count='2 %',
    subj='лютеране'
)

...

Percent(
    count='2 %',
    subj='православные'
)

Percent(
    count='2 %',
    subj='христиане'
)

Percent(
    count='1 %',
    subj='буддисты'
)

Percent(
    count='1 %',
    subj='иудеи'
)

Percent(
    count='1 %',
    subj='данных'
)

### Определения и названия политических партий

In [157]:
Party = fact(
    'Party',
    ['main']
)

PARTY = rule(
    gram('ADJF').repeatable(),
    'партия',
    TITLE.optional(), # Используем одно из написанных правил - название в кавычках
    or_(
        gram('ADJF'),
        gram('NOUN'),
    ).optional().repeatable(),
).interpretation(Party.main).interpretation(Party)

parse(PARTY)

Party(
    main='Рабочая партия'
)

Party(
    main='Социалистическая партия Франции'
)

Party(
    main='Французская социалистическая партия'
)

Party(
    main='ультраправая националистическая партия'
)

Party(
    main='Основная консервативная партия'
)

...

Party(
    main='троцкистская партия'
)

Party(
    main='Независимая рабочая партия'
)

Party(
    main='левая евроскептическая партия'
)

Party(
    main='левопопулистская евроскептическая партия'
)

Party(
    main='президентская партия «Союз за народное движение»'
)

### Деньги

In [158]:
Money = fact(
    'Money',
    ['value', 'currency']
)

Float = fact(
    'Float',
    ['value']
)


INT = yatype('INT')
FLOAT = rule(
    INT,
    ',',
    INT
)

MONEY = rule(
    rule(
        FLOAT,
        dictionary({
            'тысяча',
            'миллион',
            'млрд',
            'тыс',
            'млн'
        }).optional()
    ).interpretation(
        Money.value
    ),
    eq('евро').interpretation(
        Money.currency
    )
).interpretation(
    Money
)

parse(MONEY)

Money(
    value='1539,42',
    currency='евро'
)

Money(
    value='47,7 млрд',
    currency='евро'
)

Money(
    value='32,78 млрд',
    currency='евро'
)

Money(
    value='17,53 млрд',
    currency='евро'
)